In [119]:
import numpy as np

In [120]:
# Generate the matrix A
n = 60
A = np.diag(np.ones(n)) + np.diag(-np.ones(n-1), k=-1)
A[:, -1] = 1

In [121]:
# for i in range(n):
#     for j in range(n):
#         if(j<i):
#             A[i][j]=-1

In [122]:
print(A)

[[ 1.  0.  0. ...  0.  0.  1.]
 [-1.  1.  0. ...  0.  0.  1.]
 [ 0. -1.  1. ...  0.  0.  1.]
 ...
 [ 0.  0.  0. ...  1.  0.  1.]
 [ 0.  0.  0. ... -1.  1.  1.]
 [ 0.  0.  0. ...  0. -1.  1.]]


In [123]:
# Generate the vector x
x = np.random.rand(n)

# Compute the right-hand side vector b
b = np.dot(A, x)

In [124]:
def conjugate_gradient(A, b, x0, tol=1e-6, max_iter=1000):
    x = x0
    r = b - A @ x
    d = r.copy()
    iterations = 0
    
    for k in range(max_iter):
        alpha = np.dot(r, r) / np.dot(d, A @ d)
        x = x + alpha * d
        r_prev = r
        r = r - alpha * A @ d
        beta = np.dot(r, r) / np.dot(r_prev, r_prev)
        d = r + beta * d
        iterations += 1
        
        if np.linalg.norm(r) < tol:
            break
            
    return x


In [125]:
def jacobi(A, b, x0=None, tol=1e-6, max_iter=1000):
    n = len(b)
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0.copy()
    D = np.diag(np.diag(A))
    R = A - D
    iterations = 0
    
    for _ in range(max_iter):
        x_new = np.linalg.solve(D, b - R @ x)
        
        iterations += 1
        if np.linalg.norm(x_new - x) < tol:
            break
        
        x = x_new.copy()
    
    return x

In [126]:
def gauss_seidel(A, b, x0=None, tol=1e-6, max_iter=1000):
    b_size = len(b)
    if x0 is None:
        x = np.zeros(b_size)
    else:
        x = x0.copy()
    iterations = 0
    
    for _ in range(max_iter):
        x_new = np.zeros_like(x)
        for i in range(b_size):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        
        iterations += 1
        if np.linalg.norm(x_new - x) < tol:
            break
        
        x = x_new.copy()
    
    return x

In [127]:
def preconditioned_conjugate_gradient(A, b, x0, M, tol=1e-6, max_iter=1000):
    M_inv = np.linalg.inv(M)
    A_hat = M_inv @ A
    b_hat = M_inv @ b
    x = x0
    r = b_hat - A_hat @ x
    p = r.copy()
    iterations = 0
    
    for k in range(max_iter):
        alpha = np.dot(r, r) / np.dot(p, A_hat @ p)
        x = x + alpha * p
        r_prev = r
        r = r - alpha * A_hat @ p
        beta = np.dot(r, r) / np.dot(r_prev, r_prev)
        p = r + beta * p
        iterations += 1
        
        if np.linalg.norm(r) < tol:
            break
            
    return x, iterations

In [128]:
# Preconditioning matrix (Example: Diagonal matrix with the diagonal of A)
M = np.diag(np.diag(A))
x0 = np.zeros_like(n)

In [129]:
# Solve the linear system using different methods
x_inv = np.linalg.solve(A, b)  # Direct inversion
x_qr = np.linalg.lstsq(A, b, rcond=None)[0]  # QR decomposition solution
x_cg=conjugate_gradient(A,b, np.zeros_like(b))
x_j=jacobi(A,b)
x_g=gauss_seidel(A,b)
x_p=preconditioned_conjugate_gradient(A,b,x0,M)
x_cg

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_13336\4047450728.py:12: RuntimeWarning: overflow encountered in scalar subtract
  x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [ ]:
# Compute the 2-norm of the difference between computed vectors and true solution
diff_inv = np.linalg.norm(x_inv - x)  # Direct inversion
diff_qr = np.linalg.norm(x_qr - x)  # QR decomposition solution
diff_cg=np.linalg.norm(x_cg - x)
diff_j=np.linalg.norm(x_j - x)
diff_g=np.linalg.norm(x_g - x)
diff_p=np.linalg.norm(x_p - x)

In [ ]:
# Compute the relative differences
rel_diff_inv = diff_inv / np.linalg.norm(x)
rel_diff_qr = diff_qr / np.linalg.norm(x)
rel_diff_cg = diff_cg / np.linalg.norm(x)
rel_diff_j = diff_j / np.linalg.norm(x)
rel_diff_g = diff_g / np.linalg.norm(x)
rel_diff_p = diff_p / np.linalg.norm(x)


In [ ]:
print("Relative difference for direct inversion:", rel_diff_inv)
print("Relative difference for QR decomposition solution:", rel_diff_qr)
print("Relative difference for cg decomposition solution:", rel_diff_cg)
print("Relative difference for jaco decomposition solution:", rel_diff_j)
print("Relative difference for gause decomposition solution:", rel_diff_g)
print("Relative difference for gause decomposition solution:", rel_diff_p)


Relative difference for direct inversion: 2.0988182159919432e-16
Relative difference for QR decomposition solution: 5.775061672708036e-16
Relative difference for cg decomposition solution: 9193213.72850913
Relative difference for jaco decomposition solution: nan
Relative difference for gause decomposition solution: nan


In [ ]:
# Compute the condition number of A
condition_number = np.linalg.cond(A)
print("Condition number of matrix A:", condition_number)

Condition number of matrix A: 2.6464012577920477
